In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
cd /content/gdrive/MyDrive/Temp_Hum/

/content/gdrive/MyDrive/Temp_Hum


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

def load_multiple_csv_files(file_paths):
    all_dataframes = []

    for file_path in file_paths:
        try:
            try:
                df = pd.read_csv(file_path, encoding='cp949')
            except:
                try:
                    df = pd.read_csv(file_path, encoding='utf-8')
                except:
                    df = pd.read_csv(file_path, encoding='cp1252')

            print(f"로드된 파일: {file_path} - {len(df)} 개 데이터")
            all_dataframes.append(df)

        except Exception as e:
            print(f"파일 로드 실패 {file_path}: {e}")

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        print(f"전체 합쳐진 데이터: {len(combined_df)} 개")
        return combined_df
    else:
        print("로드된 파일이 없습니다.")
        return None

#전처리 과정
def preprocess_data(df):
    column_mapping = {}
    for col in df.columns:
        if '일시' in col or 'ÀÏ½Ã' in col:
            column_mapping[col] = 'tm'
        elif '습도' in col or '½Àµµ' in col:
            column_mapping[col] = 'hm'
        elif '기온' in col or '±â¿Â' in col:
            column_mapping[col] = 'ta'

    print("컬럼 매핑:", column_mapping)
    df = df.rename(columns=column_mapping)

    #시계열 데이터셋으로 변환
    if 'tm' in df.columns:
        try:
            df['tm'] = pd.to_datetime(df['tm'], format='%Y-%m-%d %H:%M')
        except:
            try:
                df['tm'] = pd.to_datetime(df['tm'], format='%Y-%m-%d %H:00')
            except:
                df['tm'] = pd.to_datetime(df['tm'])

    for col in ['hm', 'ta']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 시간 순으로 정렬
    df = df.sort_values('tm').reset_index(drop=True)

    # 2시간 후 습도를 타겟으로 생성
    df['hm_2h_later'] = df['hm'].shift(-2)

    df['year'] = df['tm'].dt.year
    df['month'] = df['tm'].dt.month
    df['day'] = df['tm'].dt.day
    df['hour'] = df['tm'].dt.hour
    df['dayofyear'] = df['tm'].dt.dayofyear
    df['dayofweek'] = df['tm'].dt.dayofweek

    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['sin_day'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['cos_day'] = np.cos(2 * np.pi * df['dayofyear'] / 365)

    #결측치 제거 및 이상치 필터링
    df = df.dropna(subset=['hm', 'ta', 'hm_2h_later'])
    df = df[(df['hm'] >= 0) & (df['hm'] <= 100)]

    return df

def create_features(df):
    #학습에 사용할 특성
    feature_cols = ['year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek',
                   'sin_month', 'cos_month', 'sin_hour', 'cos_hour',
                   'sin_day', 'cos_day', 'ta', 'hm']

    available_cols = [col for col in feature_cols if col in df.columns]
    df = df.dropna(subset=available_cols + ['hm_2h_later'])

    return df[available_cols], df['hm_2h_later']

def train_model(X, y):
    #훈련/테스트 = 8/2로 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    #랜덤 포레스트 모델 사용 (무난함)
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train_scaled, y_train)

    # 예측
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # 평가지표 계산
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    print(f"\n=== 모델 평가지표 ===")
    print(f"Train MSE: {train_mse:.4f} | Test MSE: {test_mse:.4f}")
    print(f"Train RMSE: {np.sqrt(train_mse):.4f} | Test RMSE: {np.sqrt(test_mse):.4f}")
    print(f"Train MAE: {train_mae:.4f} | Test MAE: {test_mae:.4f}")
    print(f"Train R²: {train_r2:.4f} | Test R²: {test_r2:.4f}")

    # 특성 중요도
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print(f"\n=== 특성 중요도 Top 10 ===")
    for i, row in feature_importance.head(10).iterrows():
        print(f"{row['feature']:15s}: {row['importance']:.4f}")

    # 예측 vs 실제 분석
    analyze_predictions(y_test, y_test_pred)

    return model, scaler, {
        'train_mse': train_mse, 'test_mse': test_mse,
        'train_r2': train_r2, 'test_r2': test_r2,
        'feature_importance': feature_importance
    }

def analyze_predictions(y_true, y_pred):
    print(f"\n=== 예측 정확도 분석 ===")

    # 오차 계산
    errors = y_true - y_pred
    abs_errors = np.abs(errors)

    print(f"평균 절대 오차: {np.mean(abs_errors):.2f}%")
    print(f"중간값 절대 오차: {np.median(abs_errors):.2f}%")
    print(f"최대 오차: {np.max(abs_errors):.2f}%")
    print(f"90% 데이터의 오차 범위: {np.percentile(abs_errors, 90):.2f}% 이내")

    # 오차 분포
    error_ranges = [
        (0, 1, "1% 이내"),
        (1, 3, "1-3%"),
        (3, 5, "3-5%"),
        (5, 10, "5-10%"),
        (10, float('inf'), "10% 초과")
    ]

    print(f"\n=== 오차 분포 ===")
    for min_err, max_err, label in error_ranges:
        if max_err == float('inf'):
            count = np.sum(abs_errors >= min_err)
        else:
            count = np.sum((abs_errors >= min_err) & (abs_errors < max_err))
        percentage = count / len(abs_errors) * 100
        print(f"{label:10s}: {count:4d}개 ({percentage:5.1f}%)")

    # 습도 구간별 성능
    humidity_ranges = [
        (0, 40, "낮음 (0-40%)"),
        (40, 60, "보통 (40-60%)"),
        (60, 80, "높음 (60-80%)"),
        (80, 100, "매우높음 (80-100%)")
    ]

    print(f"\n=== 습도 구간별 성능 ===")
    for min_hum, max_hum, label in humidity_ranges:
        mask = (y_true >= min_hum) & (y_true < max_hum)
        if np.sum(mask) > 0:
            mae = np.mean(np.abs(y_true[mask] - y_pred[mask]))
            print(f"{label:15s}: MAE {mae:.2f}% (데이터 {np.sum(mask)}개)")
        else:
            print(f"{label:15s}: 데이터 없음")

def predict_humidity(model, scaler, year, month, day, hour, temperature, current_humidity):
    from datetime import datetime

    dayofyear = datetime(year, month, day).timetuple().tm_yday
    dayofweek = datetime(year, month, day).weekday()

    sin_month = np.sin(2 * np.pi * month / 12)
    cos_month = np.cos(2 * np.pi * month / 12)
    sin_hour = np.sin(2 * np.pi * hour / 24)
    cos_hour = np.cos(2 * np.pi * hour / 24)
    sin_day = np.sin(2 * np.pi * dayofyear / 365)
    cos_day = np.cos(2 * np.pi * dayofyear / 365)

    features = np.array([[year, month, day, hour, dayofyear, dayofweek,
                         sin_month, cos_month, sin_hour, cos_hour,
                         sin_day, cos_day, temperature, current_humidity]])

    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)

    return max(0, min(100, prediction[0]))

if __name__ == "__main__":
    print("=== 습도 예측 모델 학습 ===")

    file_paths = [
        'TEMPERATURE_DATASET1.csv',
        'TEMPERATURE_DATASET2.csv',
        'TEMPERATURE_DATASET3.csv',
        'TEMPERATURE_DATASET4.csv'
    ]

    print("1. 4개 CSV 파일 로드 및 합치기...")
    df = load_multiple_csv_files(file_paths)

    if df is None or len(df) == 0:
        print("데이터 로드 실패!")
        exit()

    print("2. 데이터 전처리...")
    df_processed = preprocess_data(df)
    print(f"전처리 후 데이터: {len(df_processed)} 개")

    print("3. 특성 생성...")
    X, y = create_features(df_processed)
    print(f"특성 개수: {X.shape[1]}")
    print(f"특성 목록: {list(X.columns)}")

    print("4. 모델 학습 및 평가...")
    model, scaler, evaluation = train_model(X, y)

    print("5. 모델 저장...")
    joblib.dump(model, 'humidity_model.pkl')
    joblib.dump(scaler, 'humidity_scaler.pkl')

    print("\n=== 모델 학습 완료! ===")
    print("저장된 파일: humidity_model.pkl, humidity_scaler.pkl")

    # 모델 요약
    print(f"\n=== 최종 모델 요약 ===")
    print(f"훈련 데이터: {len(df_processed)} 개")
    print(f"특성 개수: {X.shape[1]} 개")
    print(f"테스트 정확도 (R²): {evaluation['test_r2']:.3f}")
    print(f"예측 오차 (RMSE): {np.sqrt(evaluation['test_mse']):.2f}%")

    # 가장 중요한 특성 3개
    top_features = evaluation['feature_importance'].head(3)
    print(f"\n주요 예측 인자:")
    for i, row in top_features.iterrows():
        print(f"  {i+1}. {row['feature']}: {row['importance']:.3f}")

    print("\n=== 예측 테스트 ===")
    while True:
        try:
            print("\n년, 월, 일, 시간, 현재기온, 현재습도를 입력하세요 (종료: q)")
            user_input = input("예시) 2024 12 25 14 22.5 65.0: ").strip()

            if user_input.lower() == 'q':
                break

            values = user_input.split()
            if len(values) != 6:
                print("6개 값을 모두 입력해주세요!")
                continue

            year, month, day, hour = map(int, values[:4])
            temperature = float(values[4])
            current_humidity = float(values[5])

            predicted_humidity = predict_humidity(model, scaler, year, month, day, hour, temperature, current_humidity)

            print(f"입력: {year}년 {month}월 {day}일 {hour}시, 기온 {temperature}°C, 습도 {current_humidity}%")
            print(f"2시간 후 예측 습도: {predicted_humidity:.1f}%")

        except Exception as e:
            print(f"입력 오류: {e}")
            print("올바른 형식으로 입력해주세요!")

=== 습도 예측 모델 학습 ===
1. 4개 CSV 파일 로드 및 합치기...
로드된 파일: TEMPERATURE_DATASET1.csv - 8760 개 데이터
로드된 파일: TEMPERATURE_DATASET2.csv - 8760 개 데이터
로드된 파일: TEMPERATURE_DATASET3.csv - 8760 개 데이터
로드된 파일: TEMPERATURE_DATASET4.csv - 8784 개 데이터
전체 합쳐진 데이터: 35064 개
2. 데이터 전처리...
컬럼 매핑: {'일시': 'tm', '기온(°C)': 'ta', '습도(%)': 'hm'}
전처리 후 데이터: 35062 개
3. 특성 생성...
특성 개수: 14
특성 목록: ['year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek', 'sin_month', 'cos_month', 'sin_hour', 'cos_hour', 'sin_day', 'cos_day', 'ta', 'hm']
4. 모델 학습 및 평가...

=== 모델 평가지표 ===
Train MSE: 4.0808 | Test MSE: 29.2892
Train RMSE: 2.0201 | Test RMSE: 5.4120
Train MAE: 1.3645 | Test MAE: 3.6704
Train R²: 0.9880 | Test R²: 0.9130

=== 특성 중요도 Top 10 ===
hm             : 0.8427
cos_hour       : 0.0469
ta             : 0.0189
hour           : 0.0158
sin_hour       : 0.0154
sin_day        : 0.0130
day            : 0.0111
cos_day        : 0.0099
dayofyear      : 0.0095
dayofweek      : 0.0072

=== 예측 정확도 분석 ===
평균 절대 오차: 3.67%
중간값 절대 오차: 2.5